In [3]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import math
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import json

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



#from secret import *

## WRITING THE FILES

### Firefox Driver

In [4]:
wd = webdriver.Firefox()
#driver.implicitly_wait(10)
wd.get('http://www.afl.com.au/stats')

### FUNCTION THAT PASSES IN YEAR AND ROUND AND DOWNLOADS JSON FILE

In [20]:
def data_download (_year,_round):
    movable_part = "CD_S{year}014&roundId=CD_R{year}014{round}".format(year=_year, round= _round)
    #print(movable_part)
    script = '' + \
    'window.a = jQuery.ajax({'+\
    '        url:"http://www.afl.com.au/api/cfs/afl/statsCentre/teams?competitionId={}",'.format(movable_part) +\
    '        headers: {'+\
    '                    "X-media-mis-token":"b292a40a7cfcbe808732a7ce5aa62114"'+\
    '                 }'+\
    '        });' 

    ## asks the firefox webdriver to execute this scripty
    #print (script)
    wd.execute_script(script)
    print (_year, _round)
    import time
    import json
    time.sleep(1)
    wd.execute_script("console.log('the dude', window.a.responseText);")
    result = wd.execute_script("return window.a.responseText")
    
    try:
        data = json.loads(result)
        filename= _year  + '_' + _round + '.txt'
        #with open('afl_raw/' + filename, 'w') as f:
        with open ('afl_raw/test/' + filename, 'w') as f:
            f.write(result)
            print ('exported ' + filename)
        
    except Exception as e:
        filename= _year  + '_' + _round + '.txt'
        #ith open('afl_raw/' + filename, 'w') as f:
        with open('afl_raw/test/' + filename, 'w') as f:
            f.write('error occured on writing')
            print ('export error on ' + filename)
        

            

    

### HARD CODED JQUERY TO PUT INTO CONSOLE

jQuery.ajax({url:"http://www.afl.com.au/api/cfs/afl/statsCentre/teams?competitionId=CD_S2017014&roundId=CD_R201701406", headers: {"X-media-mis-token":"b0e22835816435c8436393d14e62ecf7"}});

jQuery.ajax({url:" http://www.afl.com.au/api/cfs/afl/statsCentre/teams?competitionId=CD_S2003014&roundId=CD_R200301401", headers: {"X-media-mis-token":"b0e22835816435c8436393d14e62ecf7"}});

### LIST OF POSSIBLE YEARS AND ROUNDS FOR DATA

In [171]:
year_list = ['2017','2016','2015','2014','2013','2012','2011','2010','2009','2008','2007','2006','2005','2004','2003','2002', '2001']
round_list = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27']

### RUN A LOOP OVER ALL ROUNDS IN ALL YEARS TO INITIATE THE DOWNLOAD FUNCTION

In [22]:
for i in year_list:
    current_year = i
    for i in round_list:
        current_round = i
        data_download(current_year,current_round)

2017 01
exported 2017_01.txt
2017 02
exported 2017_02.txt
2017 03
exported 2017_03.txt
2017 04
exported 2017_04.txt
2017 05
exported 2017_05.txt
2017 06
exported 2017_06.txt
2017 07
exported 2017_07.txt
2017 08
export error on 2017_08.txt
2017 09
export error on 2017_09.txt
2017 10
export error on 2017_10.txt
2017 11
export error on 2017_11.txt
2017 12
export error on 2017_12.txt
2017 13


KeyboardInterrupt: 

### TESTING INDIVIDUAL DOWNLOADS

current_year = '2005'
current_round = '05'

data_download(current_year,current_round)

## LOAD FILES IN PYTHON

### CREATE DICTIONARY FOR MATCH ID'S

In [172]:
match_dict= {0:'01',1:'01',2:'02',3:'02', 4:'03',5:'03', 6:'04',7:'04',8:'05',9:'05', 10:'06',11:'06',12:'07',13:'07', 14:'08',15:'08', 16:'09',17:'09'}
print (match_dict[3])

02


### MERGING TWO DICTIONARIES AND ADDING TO DATAFRAME

a = {"b":1, "c": 2}

d = {"b":3, "c": 2}

x = [a,d]

pd.DataFrame(x)



z = {**x, **y}
http://stackoverflow.com/questions/38987/how-to-merge-two-python-dictionaries-in-a-single-expression

### FUNCTION TO LOOK AT EACH MATCH, SORT AND  ASSIGN A MATCH_ID

In [220]:
def structure_data(d, i, _year, _round):
    team_stats= d['team']
    match_stats = d['stats']['totals']
    # ** merges two dictionaries together
    stats = {**team_stats, **match_stats}
    stats['year'] = _year
    stats['round'] = _round
    # Finding match_id
    #string formatting mini lanuage
    match_dict= {0:'01',1:'01',2:'02',3:'02', 4:'03',5:'03', 6:'04',7:'04',8:'05',9:'05', 10:'06',11:'06',12:'07',13:'07', 14:'08',15:'08', 16:'09',17:'09'}
    #match_number = "{0:0=2d}".format(int(math.ceil(i/2)*2))
    match_number = (match_dict[i])
    stats['match_id'] = 'CD_M' + _year + '014'+  _round + str(match_number)
    #CD_M20170140401
    # W know home is the evan numbers in the list- so if there is no remainder when divived by two its divisible by 2
    stats['home'] = i % 2 == 0
    stats = {**stats, **stats['clearances']}
    stats.pop('clearances', None)
    #try:
        #stats = {**stats, **stats['interchangeCounts']}
        #stats.pop('interchangeCounts', None)
   #except:
        #print (stats)
        # interchange counts are equal to zero
    return stats

### FUNCTION TO READ EACH DATA FILE- RETURNS A RELEVENT LIST OF ROUNDS GAMES

In [221]:
def readfile(_year, _round):
    filename= _year  + '_' + _round + '.txt'
    path = 'afl_raw/' + filename
    print (current_year, current_round, path)
    with open(path) as f:
        json_string = f.read()
        json_string = json_string.strip()
        if json_string != "error occured on writing":
            data = json.loads(json_string)
            this_round = []
            for i, d in enumerate(data['lists']):
                stats = structure_data(d, i,_year, _round)
                this_round.append(stats)
            return this_round

### PUT THE ROUND LIST INTO A DATA FRAME

In [222]:
mega_list = []
for current_year in year_list:
    for current_round in round_list:
        data = readfile(current_year, current_round)
        if data:
            try:
                mega_list.extend(data)
            except Exception as e:
                print(e, data)
            #print (current_year)
        #except Exception as e:
            #print (e)


2017 01 afl_raw/2017_01.txt
2017 02 afl_raw/2017_02.txt
2017 03 afl_raw/2017_03.txt
2017 04 afl_raw/2017_04.txt
2017 05 afl_raw/2017_05.txt
2017 06 afl_raw/2017_06.txt
2017 07 afl_raw/2017_07.txt
2017 08 afl_raw/2017_08.txt
2017 09 afl_raw/2017_09.txt
2017 10 afl_raw/2017_10.txt
2017 11 afl_raw/2017_11.txt
2017 12 afl_raw/2017_12.txt
2017 13 afl_raw/2017_13.txt
2017 14 afl_raw/2017_14.txt
2017 15 afl_raw/2017_15.txt
2017 16 afl_raw/2017_16.txt
2017 17 afl_raw/2017_17.txt
2017 18 afl_raw/2017_18.txt
2017 19 afl_raw/2017_19.txt
2017 20 afl_raw/2017_20.txt
2017 21 afl_raw/2017_21.txt
2017 22 afl_raw/2017_22.txt
2017 23 afl_raw/2017_23.txt
2017 24 afl_raw/2017_24.txt
2017 25 afl_raw/2017_25.txt
2017 26 afl_raw/2017_26.txt
2017 27 afl_raw/2017_27.txt
2016 01 afl_raw/2016_01.txt
2016 02 afl_raw/2016_02.txt
2016 03 afl_raw/2016_03.txt
2016 04 afl_raw/2016_04.txt
2016 05 afl_raw/2016_05.txt
2016 06 afl_raw/2016_06.txt
2016 07 afl_raw/2016_07.txt
2016 08 afl_raw/2016_08.txt
2016 09 afl_raw/2016

### TEST DATAFRAME AND REVIEW SIZE

In [223]:
df = pd.DataFrame(mega_list)
print (df.shape)
df.head(300)
df.shape

(6252, 48)


(6252, 48)

### SEE DATA TYPES

In [225]:
df.dtypes

behinds                   float64
bounces                   float64
centreClearances          float64
clangers                  float64
contestedMarks            float64
contestedPossessions      float64
disposalEfficiency        float64
disposals                 float64
dreamTeamPoints           float64
freesAgainst              float64
freesFor                  float64
goalAccuracy              float64
goalAssists               float64
goalEfficiency            float64
goals                     float64
handballs                 float64
hitouts                   float64
home                         bool
inside50s                 float64
intercepts                float64
interchangeCounts          object
kicks                     float64
lastUpdated                object
marks                     float64
marksInside50             float64
match_id                   object
metresGained               object
onePercenters             float64
ranking                    object
ratingPoints  

### ASSIGN OPPONENT- LIST TECHNIQUE

grades = []

for index, row in df.iterrows():
    current_name = row['teamName']
    current_match = row['match_id']
    print  (current_name)
    for index_, row_ in df.iterrows():
        if (row_['match_id'] == current_match) & (row_['teamName'] != current_name):
            opponent = row_['teamName']
            grades.append(opponent)
            print (opponent)
            

df['opponent'] = grades

### ASSIGN OPPONENT- VECTORISED TECHNIQUE

In [226]:
def opponent_(x):
    game_pair = df[df.match_id == x.match_id]
    other_team = game_pair[game_pair.teamName != x.teamName]
    return list(other_team.teamName)[0]

In [227]:
df['Opponent']= df.apply(opponent_, axis =1)

### TOTAL SCORE

In [228]:
def total_(x):
    total_score = x.behinds + ((x.goals)*6)                   
    return total_score

In [229]:
df['total_score']= df.apply(total_, axis =1)

### ASSIGN WIN OR LOSS- VECTORISED TECHNIQUE

In [230]:
def win_(x):
    x_score = x.total_score
    #Pair up with same match id
    game_pair = df[x.match_id == df.match_id]
    # Find the other team
    other_team = game_pair[game_pair.teamName != x.teamName]
    # Find other team score
    other_score = list(other_team.total_score)[0]
    #return (other_score)
    if x_score > other_score:
        return 'Win'
    elif x_score == other_score:
        return 'Draw'
    else: 
        return "Lose"

In [231]:
df['Result']= df.apply(win_, axis =1)

### COLUMN HEADERS CAPITALISED

In [232]:
df.columns = map(str.upper, df.columns)

### TEST IN HEAD

In [235]:
df.head()
df.shape

(6252, 51)

### TEST A SPECPIFIC ROWS RESULT

df.loc[0].total_score

### EXPORTING TO CSV (Create stable copy for web)

In [236]:
df.to_csv("afl_data_set_0.csv")

## WORK WITH STABLE CSV

### LOAD IN CSV FOR STABLE FILE

In [237]:
if os.path.isfile("afl_data_set_0.csv"):
    filepath = "afl_data_set_0.csv"
    print ("loading from file")
else:
    filepath = "http://www.osr.nsw.gov.au/sites/default/files/file_manager/penalty_data_set_0.csv"
    print ("loading from the internet")

df = pd.read_csv(filepath)
print ("done")

loading from file
done


### TEST DATAFRAME

In [239]:
df.head()
#df.shape

,Unnamed: 0,BEHINDS,BOUNCES,CENTRECLEARANCES,CLANGERS,CONTESTEDMARKS,CONTESTEDPOSSESSIONS,DISPOSALEFFICIENCY,DISPOSALS,DREAMTEAMPOINTS,...,TEAMNAME,TEAMNICKNAME,TOTALCLEARANCES,TOTALPOSSESSIONS,TURNOVERS,UNCONTESTEDPOSSESSIONS,YEAR,OPPONENT,TOTAL_SCORE,RESULT
0,0,5.0,8.0,19.0,49.0,14.0,138.0,73.6,348.0,1604.0,...,Carlton,Blues,38.0,341.0,81.0,203.0,2017,Richmond,89.0,Lose
1,1,12.0,3.0,16.0,55.0,11.0,149.0,79.0,352.0,1551.0,...,Richmond,Tigers,40.0,354.0,68.0,205.0,2017,Carlton,132.0,Win
2,2,14.0,2.0,16.0,45.0,12.0,161.0,74.5,431.0,1835.0,...,Collingwood,Magpies,42.0,437.0,70.0,276.0,2017,Western Bulldogs,86.0,Lose
3,3,10.0,3.0,10.0,44.0,12.0,135.0,77.0,392.0,1774.0,...,Western Bulldogs,Bulldogs,26.0,394.0,63.0,259.0,2017,Collingwood,100.0,Win
4,4,12.0,6.0,14.0,51.0,9.0,139.0,69.7,337.0,1341.0,...,St Kilda,Saints,31.0,338.0,62.0,199.0,2017,Melbourne,90.0,Lose


## ANALYSE

### DRAW GRAPHS IN THE NOTEBOOK

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)